# Import library


In [1]:
from google.colab import userdata
hf_token = userdata.get("HUGGINGFACE_TOKEN")
from huggingface_hub import login
login(hf_token)

In [2]:
!pip install langchain_community langchainhub chromadb langchain FlagEmbedding bitsandbytes

In [3]:
import pandas as pd
import uuid
import re
import json
import os
from operator import itemgetter
from tqdm.notebook import tqdm
from typing import List, Dict
from langchain.schema.document import Document
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains.summarize import load_summarize_chain
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.load import dumps, loads
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnableLambda

In [4]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_token

In [5]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# LLM

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
model_id = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_size = "right"

In [8]:
import transformers
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=False,
    max_new_tokens=64,
    temperature=0.2
)

Device set to use cuda:0


In [9]:
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=generate_text)

<ipython-input-9-6f68aebbe00e>:2: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=generate_text)


# Loading data

In [ ]:
!pip install unstructured[all-docs]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.1/526.1 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!sudo apt-get install poppler-utils tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  poppler-utils tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 4 newly installed, 0 to remove and 29 not upgraded.
Need to get 5,002 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 5,002 kB in 1s (3,748 kB/s)
debconf: unable to initialize frontend: Dial

In [ ]:
data_path = "/content/drive/MyDrive/[TA] Falcon/Lesson 4/data/Luật giao thông.pdf"

In [ ]:
from lxml import html
from pydantic import BaseModel
from typing import Any, Optional
from unstructured.partition.pdf import partition_pdf
from tqdm import tqdm

elements = partition_pdf(filename=data_path,
                          extract_images_in_pdf=False,
                          infer_table_structure=True,
                          strategy='hi_res',
                        )

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [ ]:
category_counts = {}
for element in elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.Header'>": 1,
 "<class 'unstructured.documents.elements.Title'>": 56,
 "<class 'unstructured.documents.elements.NarrativeText'>": 421,
 "<class 'unstructured.documents.elements.ListItem'>": 1,
 "<class 'unstructured.documents.elements.Image'>": 1}

In [ ]:
class Element(BaseModel):
    type: str
    text: Any

# Categorize by type
categorized_elements = []
text_types = ["<class 'unstructured.documents.elements.Title'>",
        "<class 'unstructured.documents.elements.Text'>",
        "<class 'unstructured.documents.elements.NarrativeText'>"]
for element in elements:
    if str(type(element)) in text_types:
        categorized_elements.append(Element(type="text", text=str(element)))
# Text
text_elements = [e for e in categorized_elements if e.type == "text"]
print(len(text_elements))

477


In [ ]:
text_elements[15:20]

[Element(type='text', text='1. Đường bộ gồm đường, cầu đường bộ, hầm đường bộ, bến phà đường bộ.'),
 Element(type='text', text='2. Công trình đường bộ gồm đường bộ, nơi dừng xe, đỗ xe trên đường bộ, đèn tín hiệu, biển báo hiệu, vạch kẻ đường, cọc tiêu, rào chắn, đảo giao thông, dải phân cách, cột cây số, tường, kè, hệ thống thoát nước, trạm kiểm tra tải trọng xe, trạm thu phí và các công trình, thiết bị phụ trợ đường bộ khác.'),
 Element(type='text', text='3. Kết cấu hạ tầng giao thông đường bộ gồm công trình đường bộ, bến xe, bãi đỗ xe, trạm dừng nghỉ và các công trình phụ trợ khác trên đường bộ phục vụ giao thông và hành lang an toàn đường bộ.'),
 Element(type='text', text='4. Đất của đường bộ là phần đất trên đó công trình đường bộ được xây dựng và phần đất dọc hai bên đường bộ để quản lý, bảo trì, bảo vệ công trình đường bộ.'),
 Element(type='text', text='5. Hành lang an toàn đường bộ là dải đất dọc hai bên đất của đường bộ, tính từ mép ngoài đất của đường bộ ra hai bên để bảo đảm 

In [ ]:
with open('/content/drive/MyDrive/[TA] Falcon/Lesson 4/data/text_elements.json', 'w') as f:
  json.dump([e.dict() for e in text_elements], f, indent=4, ensure_ascii=False)

<ipython-input-61-f311ef899de4>:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  json.dump([e.dict() for e in text_elements], f, indent=4, ensure_ascii=False)


# Document Splitting

## Pre-processing

In [ ]:
import json
with open('/content/drive/MyDrive/[TA] Falcon/Lesson 4/data/text_elements.json', 'r') as f:
  text_elements = json.load(f)

In [ ]:
all_sentences = [element["text"] for element in text_elements]
len(all_sentences)

477

In [ ]:
import re
def contains_letters(text):
  pattern = re.compile(r'[a-zA-Z]')
  return bool(pattern.search(text))

new_all_sentences = []
for sent in all_sentences:
  if len(sent) > 10 and contains_letters(sent):
    new_all_sentences.append(sent)
all_sentences = new_all_sentences

In [ ]:
len(all_sentences)

475

## Semantic Chunking

In [ ]:
sentences = [{'sentence': x, 'index': i} for i, x in enumerate(all_sentences)]
sentences[15:18]

[{'sentence': '2. Công trình đường bộ gồm đường bộ, nơi dừng xe, đỗ xe trên đường bộ, đèn tín hiệu, biển báo hiệu, vạch kẻ đường, cọc tiêu, rào chắn, đảo giao thông, dải phân cách, cột cây số, tường, kè, hệ thống thoát nước, trạm kiểm tra tải trọng xe, trạm thu phí và các công trình, thiết bị phụ trợ đường bộ khác.',
  'index': 15},
 {'sentence': '3. Kết cấu hạ tầng giao thông đường bộ gồm công trình đường bộ, bến xe, bãi đỗ xe, trạm dừng nghỉ và các công trình phụ trợ khác trên đường bộ phục vụ giao thông và hành lang an toàn đường bộ.',
  'index': 16},
 {'sentence': '4. Đất của đường bộ là phần đất trên đó công trình đường bộ được xây dựng và phần đất dọc hai bên đường bộ để quản lý, bảo trì, bảo vệ công trình đường bộ.',
  'index': 17}]

In [ ]:
def combine_sentence(sentences, buffer_size=1):
  for i in range(len(sentences)):
    combined_sentence = ''
    for j in range(i - buffer_size, i):
      if j >= 0:
        combined_sentence += sentences[j]['sentence'] + ' '
    combined_sentence += sentences[i]['sentence']
    for j in range(i + 1, i + 1 + buffer_size):
      if j < len(sentences):
        combined_sentence += ' ' + sentences[j]['sentence']
    sentences[i]['combined_sentence'] = combined_sentence
  return sentences
sentences = combine_sentence(sentences)

In [ ]:
sentences[9]

{'sentence': 'Luật này quy định về quy tắc giao thông đường bộ; kết cấu hạ tầng giao thông đường bộ; phương tiện và người tham gia giao thông đường bộ; vận tải đường bộ và quản lý nhà nước về giao thông đường bộ.',
 'index': 9,
 'combined_sentence': 'Điều 1. Phạm vi điều chỉnh Luật này quy định về quy tắc giao thông đường bộ; kết cấu hạ tầng giao thông đường bộ; phương tiện và người tham gia giao thông đường bộ; vận tải đường bộ và quản lý nhà nước về giao thông đường bộ. Điều 2. Đối tượng áp dụng'}

In [ ]:
from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer('keepitreal/vietnamese-sbert')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embedding = embed_model.encode([x['combined_sentence'] for x in sentences])
for i, sentence in enumerate(sentences):
  sentence['combined_sentence_embedding'] = embedding[i]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def calculate_cosine_distance(sentences):
  distances = []
  for i in range(len(sentences) - 1):
    embedding_cur = sentences[i]['combined_sentence_embedding']
    embedding_next = sentences[i + 1]['combined_sentence_embedding']
    similarity = cosine_similarity([embedding_cur], [embedding_next])[0][0]
    distance = 1 - similarity
    distances.append(distance)
    sentences[i]['distance_to_next'] = distance
  return distances, sentences
distances, sentences = calculate_cosine_distance(sentences)

In [ ]:
import numpy as np
#Phân vị
breakpoint_percentile_threshold = 80
breakpoint_distance_threshold = np.percentile(distances, breakpoint_percentile_threshold)
num_distances_above_theshold = len([x for x in distances if x > breakpoint_distance_threshold])

indices_above_thresh = [i for i, x in enumerate(distances) if x > breakpoint_distance_threshold]

In [ ]:
start_index = 0
chunks = []
for index in indices_above_thresh:
  end_index = index
  group = sentences[start_index: end_index + 1]
  combined_text = ' '.join(d['sentence'] for d in group)
  chunks.append(combined_text)
  start_index = end_index + 1
if start_index < len(sentences):
  combined_text = ' '.join(d['sentence'] for d in sentences[start_index:])
  chunks.append(combined_text)

In [ ]:
chunks[10]

'1. Hoạt động giao thông đường bộ phải bảo đảm thông suốt, trật tự, an toàn, hiệu quả; góp phần phát triển kinh tế - xã hội, bảo đảm quốc phòng, an ninh và bảo vệ môi trường. 2. Phát triển giao thông đường bộ theo quy hoạch, từng bước hiện đại và đồng bộ; gắn kết phương thức vận tải đường bộ với các phương thức vận tải khác. 3. Quản lý hoạt động giao thông đường bộ được thực hiện thống nhất trên cơ sở phân công, phân cấp trách nhiệm, quyền hạn cụ thể, đồng thời có sự phối hợp chặt chẽ giữa các bộ, ngành và chính quyền địa phương các cấp. 4. Bảo đảm trật tự, an toàn giao thông đường bộ là trách nhiệm của cơ quan, tổ chức, cá nhân. 5. Người tham gia giao thông phải có ý thức tự giác, nghiêm chỉnh chấp hành quy tắc giao thông, giữ gìn an toàn cho mình và cho người khác. Chủ phương tiện và người điều khiển phương tiện phải chịu trách nhiệm trước pháp luật về việc bảo đảm an toàn của phương tiện tham gia giao thông đường bộ. 6. Mọi hành vi vi phạm pháp luật giao thông đường bộ phải được phá

## Chunk with title

In [ ]:
all_chunks = []
i = 0
while i < len(all_sentences):
  first_sentence = all_sentences[i]
  chunk = first_sentence
  if(first_sentence.startswith("Điều")):
    # Duyệt thêm các câu tiếp theo
    i += 1
    while i < len(all_sentences):
      # Break nếu len(chunk) > 1000 hoặc sentence là Điều tiếp theo
      if(len(chunk) > 1000) or all_sentences[i].startswith("Điều "):
        break
      chunk += " " + all_sentences[i]
      i += 1
  # Bắt đầu bằng khoản
  elif re.match(r'^\d+\.\s', first_sentence):
    # TÌm kiếm Điều của khoản này
    for j in range(i - 1, -1, -1):
      if all_sentences[j].startswith("Điều"):
        chunk = all_sentences[j] + " " + chunk + " "
        break
    # Duyệt thêm các câu tiếp theo
    i += 1
    while i < len(all_sentences):
      # Break nếu len(chunk) > 1000 hoặc sentence là 1 Điều, khoản khác
      if(len(chunk) > 1000) or all_sentences[i].startswith("Điều ") or re.match(r'^\d+\.\s', all_sentences[i]):
        break
      chunk += all_sentences[i] + " "
      i += 1
  elif re.match(r'^[a-zđ]\)\s', first_sentence):
    # Tìm kiếm Điều, khoản của ý này
    for j in range(i - 1, -1, -1):
      if re.match(r'^\d+\.\s', all_sentences[j]):
        chunk = all_sentences[j] + " " + chunk + " "
        break
    for k in range(i - 1, -1, -1):
      if all_sentences[k].startswith("Điều"):
        chunk = all_sentences[k] + " " + chunk + " "
        break
    # Duyệt thêm các câu tiếp theo
    i += 1
    while i < len(all_sentences):
      # Break nếu len(chunk) > 1000 hoặc sentence là 1 Điều, khoản tiếp theo
      if(len(chunk) > 1000) or all_sentences[i].startswith("Điều ") or re.match(r'^\d+\.\s', all_sentences[i]):
        break
      chunk += all_sentences[i] + " "
      i += 1
  else:
    # Các thông tin bên lề
    while i < len(all_sentences):
      if(all_sentences[i].startswith("Điều ") or re.match(r'^\d+\.\s', all_sentences[i]) or re.match(r'^[a-zđ]\)\s', all_sentences[i])):
        break
      chunk += all_sentences[i] + " "
      i += 1
  all_chunks.append(chunk)

In [ ]:
all_chunks[10]

'Điều 3. Giải thích từ ngữ 10. Dải phân cách là bộ phận của đường để phân chia mặt đường thành hai chiều xe chạy riêng biệt hoặc để phân chia phần đường của xe cơ giới và xe thô sơ. Dải phân cách gồm loại cố định và loại di động. '

In [ ]:
all_chunks[12]

'Điều 3. Giải thích từ ngữ 12. Đường cao tốc là đường dành cho xe cơ giới, có dải phân cách chia đường cho xe chạy hai chiều riêng biệt; không giao nhau cùng mức với một hoặc các đường khác; được bố trí đầy đủ trang thiết bị phục vụ, bảo đảm giao thông liên tục, an toàn, rút ngắn thời gian hành trình và chỉ cho xe ra, vào ở những điểm nhất định. '

In [ ]:
text_documents = [Document(page_content=chunk) for chunk in all_chunks]

In [ ]:
with open('/content/drive/MyDrive/[TA] Falcon/Lesson 4/data/chunk_w_title.json', 'w') as f:
  json.dump([e.dict() for e in text_documents], f, indent=4, ensure_ascii=False)

<ipython-input-59-40208c7e4747>:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  json.dump([e.dict() for e in text_documents], f, indent=4, ensure_ascii=False)


# Indexing

In [10]:
with open('/content/drive/MyDrive/[TA] Falcon/Lesson 4/Lab 03/data/chunk_w_title.json', 'r') as f:
  text_dict = json.load(f)
chunks = [Document(page_content=doc["page_content"]) for doc in text_dict]

In [18]:
summary_prompt = PromptTemplate(
    input_variables=["text"],
    template = """Tóm tắt ngắn gọn nội dung sau (không quá 100 từ), vẫn giữ nguyên ý chính, không thêm thông tin không có trong văn bản:\n\n{text}\n\nTóm tắt:"""
)

chain = load_summarize_chain(llm, prompt=summary_prompt)
# Tóm tắt câu dài hơn đi
doc = Document(page_content="""Điều 6a. Quy hoạch kết cấu hạ tầng giao thông đường bộ3 1. Quy hoạch kết cấu hạ tầng giao thông đường bộ là quy hoạch có tính chất kỹ thuật, chuyên ngành, cụ thể hóa quy hoạch mạng lưới đường bộ, xác định phương án phát triển công trình đường bộ và các kết cấu hạ tầng giao thông đường bộ khác theo từng tuyến đường bộ. 2. Quy hoạch kết cấu hạ tầng giao thông đường bộ bao gồm các nội dung chủ yếu sau đây: a) Xác định hướng tuyến, các điểm khống chế chính, chiều dài, quy mô các tuyến đường bộ qua từng địa phương, từng vùng; xác định số lượng, quy mô, thông số kỹ thuật chủ yếu của các công trình chính gồm cầu, hầm, bến phà trên tuyến đường bộ; xác định cụ thể các điểm giao cắt, hệ thống trạm dừng nghỉ, công trình phụ trợ khác; b) Phương án kết nối với các phương thức vận tải đường sắt, đường thủy nội địa, hàng hải, hàng không cho từng khu vực, từng tuyến đường; kết nối với hệ thống đô thị, khu kinh tế, khu du lịch, khu công nghiệp, khu chế xuất; c) Xác định nhu cầu sử dụng đất, nhu cầu vốn đầu tư, lộ trình thực hiện quy hoạch theo thứ tự ưu tiên đầu tư;""")
chain.invoke({"input_documents": [doc]})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'input_documents': [Document(metadata={}, page_content='Điều 6a. Quy hoạch kết cấu hạ tầng giao thông đường bộ3 1. Quy hoạch kết cấu hạ tầng giao thông đường bộ là quy hoạch có tính chất kỹ thuật, chuyên ngành, cụ thể hóa quy hoạch mạng lưới đường bộ, xác định phương án phát triển công trình đường bộ và các kết cấu hạ tầng giao thông đường bộ khác theo từng tuyến đường bộ. 2. Quy hoạch kết cấu hạ tầng giao thông đường bộ bao gồm các nội dung chủ yếu sau đây: a) Xác định hướng tuyến, các điểm khống chế chính, chiều dài, quy mô các tuyến đường bộ qua từng địa phương, từng vùng; xác định số lượng, quy mô, thông số kỹ thuật chủ yếu của các công trình chính gồm cầu, hầm, bến phà trên tuyến đường bộ; xác định cụ thể các điểm giao cắt, hệ thống trạm dừng nghỉ, công trình phụ trợ khác; b) Phương án kết nối với các phương thức vận tải đường sắt, đường thủy nội địa, hàng hải, hàng không cho từng khu vực, từng tuyến đường; kết nối với hệ thống đô thị, khu kinh tế, khu du lịch, khu công nghiệp, k

In [11]:
DB_PATH = "/content/drive/MyDrive/[TA] Falcon/Lesson 4/Lab 03/data/chroma_db"
SUMMARY_PATH = "/content/drive/MyDrive/[TA] Falcon/Lesson 4/Lab 03/data/summaries.json"

def summary_document(chunks: List[Document], summary_path=SUMMARY_PATH) -> List[Document]:
  if os.path.exists(summary_path):
    with open(summary_path, "r", encoding="utf-8") as f:
      summaries_data = json.load(f)

    return [Document(page_content=s['text'], metadata=s['metadata']) for s in summaries_data]

  summary_prompt = PromptTemplate(
      input_variables=["text"],
      template = """Tóm tắt ngắn gọn nội dung sau, giữ nguyên ý chính và không thêm thông tin không có trong văn bản:\n\n{text}\n\nTóm tắt:"""
  )
  chain = load_summarize_chain(llm, prompt=summary_prompt)

  summaries = []
  summaries_data = []
  for doc in tqdm(chunks):
    unique_id = str(uuid.uuid4())
    chunk_summary = chain.invoke({"input_documents": [doc]})
    summary_text = chunk_summary['output_text']
    summary_doc = Document(page_content=summary_text, metadata={"doc_id": unique_id})
    summaries.append(summary_doc)
    summaries_data.append({"text": summary_text, "metadata": {"doc_id": unique_id}})

  with open(summary_path, "w", encoding="utf-8") as f:
    json.dump(summaries_data, f, ensure_ascii=False, indent=4)
  return summaries

In [26]:
summaries = summary_document(chunks)

  0%|          | 0/135 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_to

In [12]:
def create_retriever(summaries, chunks, db_path=DB_PATH):
  embedding = SentenceTransformerEmbeddings(model_name="keepitreal/vietnamese-sbert")
  if os.path.exists(db_path):
    vectorstore = Chroma(persist_directory=db_path, embedding_function=embedding)
  else:
    vectorstore = Chroma.from_documents(
        documents=summaries,
        embedding=embedding,
        persist_directory=db_path
    )
    vectorstore.persist()

  for summary, doc in zip(summaries, chunks):
    doc.metadata["doc_id"] = summary.metadata["doc_id"]

  docstore = InMemoryStore()
  retriever = MultiVectorRetriever(
      vectorstore=vectorstore,
      docstore=docstore,
      id_key = "doc_id",
      search_kwargs={"k": 4}
  )
  retriever.docstore.mset([(doc.metadata["doc_id"], doc) for doc in chunks])
  return retriever

In [30]:
retriever = create_retriever(summaries, chunks)

<ipython-input-29-4292f3ec9d0a>:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory=db_path, embedding_function=embedding)


In [31]:
retrieved_docs = retriever.get_relevant_documents("Hệ thống báo hiệu đường bộ gồm những gì")
for document in retrieved_docs:
  print(document.page_content)

Điều 10. Hệ thống báo hiệu đường bộ 1. Hệ thống báo hiệu đường bộ gồm hiệu lệnh của người điều khiển giao thông; tín hiệu đèn giao thông, biển báo hiệu, vạch kẻ đường, cọc tiêu hoặc tường bảo vệ, rào chắn. 2. Hiệu lệnh của người điều khiển giao thông quy định như sau: a) Tay giơ thẳng đứng để báo hiệu cho người tham gia giao thông ở các hướng dừng lại; b) Hai tay hoặc một tay dang ngang để báo hiệu cho người tham gia giao thông ở phía trước và ở phía sau người điều khiển giao thông phải dừng lại; người tham gia giao thông ở phía bên phải và bên trái của người điều khiển giao thông được đi; c) Tay phải giơ về phía trước để báo hiệu cho người tham gia giao thông ở phía sau và bên phải người điều khiển giao thông phải dừng lại; người tham gia giao thông ở phía trước người điều khiển giao thông được rẽ phải; người tham gia giao thông ở phía bên trái người điểu khiển giao thông được đi tất cả các hướng; người đi bộ qua đường phải đi sau lưng người điều khiển giao thông. 3. Tín hiệu đèn giao

<ipython-input-31-0a83423b1d14>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents("Hệ thống báo hiệu đường bộ gồm những gì")


# Re-rank

In [13]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('namdp-ptit/ViRanker', user_fp16=True)

In [14]:
def rerank_documents(results, question):
  reranked_results = []
  for doc in results:
    doc_str = doc.page_content
    similarity_score = reranker.compute_score([question, doc_str], normalize=True)[0]
    if similarity_score > 0.7:
      reranked_results.append((doc_str, similarity_score))
  reranked_results = sorted(reranked_results, key=lambda x: x[1], reverse=True)
  reranked_results = [x[0] for x in reranked_results]
  return reranked_results

In [36]:
question = "Hệ thống báo hiệu đường bộ gồm những gì?"
retrieved_docs = retriever.get_relevant_documents(question)
reranked_docs = rerank_documents(retrieved_docs, question)
for document in reranked_docs:
  print(document)

Điều 10. Hệ thống báo hiệu đường bộ 1. Hệ thống báo hiệu đường bộ gồm hiệu lệnh của người điều khiển giao thông; tín hiệu đèn giao thông, biển báo hiệu, vạch kẻ đường, cọc tiêu hoặc tường bảo vệ, rào chắn. 2. Hiệu lệnh của người điều khiển giao thông quy định như sau: a) Tay giơ thẳng đứng để báo hiệu cho người tham gia giao thông ở các hướng dừng lại; b) Hai tay hoặc một tay dang ngang để báo hiệu cho người tham gia giao thông ở phía trước và ở phía sau người điều khiển giao thông phải dừng lại; người tham gia giao thông ở phía bên phải và bên trái của người điều khiển giao thông được đi; c) Tay phải giơ về phía trước để báo hiệu cho người tham gia giao thông ở phía sau và bên phải người điều khiển giao thông phải dừng lại; người tham gia giao thông ở phía trước người điều khiển giao thông được rẽ phải; người tham gia giao thông ở phía bên trái người điểu khiển giao thông được đi tất cả các hướng; người đi bộ qua đường phải đi sau lưng người điều khiển giao thông. 3. Tín hiệu đèn giao

# Combine all the steps into one function

In [18]:
def get_context(inputs: Dict[str, str]) -> Dict[str, str]:
  question = inputs["question"]
  chunks = inputs["chunks"]
  summaries = summary_document(chunks)
  retriever = create_retriever(summaries, chunks)
  retrieved_docs = retriever.get_relevant_documents(question)
  reranked_docs = rerank_documents(retrieved_docs, question)
  return {"question": question, "context": reranked_docs}

# RAG chain


In [19]:
template = """Bạn là chuyên gia về Luật Giao thông đường bộ Việt Nam. Hãy trả lời một cách chính xác, ngắn gọn không lặp lại thông tin và không tạo ra gì thêm ngoài câu trả lời.
    ### **Ví dụ**
    **Câu hỏi:** "Quy định về chấp hành tín hiệu đèn giao thông là gì?"
    **Trả lời:** "Tín hiệu đèn giao thông có ba màu: xanh (được đi), đỏ (dừng lại), vàng (chuẩn bị dừng, có thể đi nếu đã quá vạch dừng)." \n\n
    ---

    Bây giờ, trả lời câu hỏi dựa trên ngữ cảnh sau:
    {context}

    {question}
    **Trả lời:**
"""

prompt = ChatPromptTemplate.from_template(template)
def generate_answer(question):
  final_rag_chain = (
      RunnableLambda(get_context)
      | prompt
      | llm
      | StrOutputParser()
  )
  answer = final_rag_chain.invoke({"question": question, "chunks":chunks})
  return answer

In [21]:
question1 = "Hệ thống báo hiệu đường bộ gồm những gì"
print(generate_answer(question1))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


    Hệ thống báo hiệu đường bộ gồm hiệu lệnh của người điều khiển giao thông; tín hiệu đèn giao thông, biển báo hiệu, vạch kẻ đường, cọc tiêu hoặc tường bảo vệ, rào chắn.


# Chat


In [22]:
end_text = ["tạm biệt", "chào tạm biệt"]
def rag_chat():
  user_input = ""
  separated = '-' * 100
  while(user_input.lower() not in end_text):
    if(user_input == ""):
      user_input = input("| You |\n-------\n")
    if(user_input.lower() not in end_text):
      print(f'{separated}\n| Model |\n---------\n')
      output = generate_answer(user_input)
      # start_idx = output.find("Trả lời:") + len("Trả lời")
      # result = output[start_idx:]
      print(output.strip())
      user_input = ""
      print(f'{separated}\n')
      user_input = input("| You |\n-------\n")
    else:
      break

In [23]:
rag_chat()

| You |
-------
Xin chào, bạn là ai?
----------------------------------------------------------------------------------------------------
| Model |
---------



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Tôi là một chuyên gia về Luật Giao thông đường bộ Việt Nam. Tôi có thể giúp bạn hiểu rõ hơn về các quy định và nguyên tắc liên quan đến giao thông đường bộ tại Việt Nam. Bạn có thể đặt câu hỏi cụ thể nào đó liên quan đến Luật Giao thông đường bộ Việt Nam không
----------------------------------------------------------------------------------------------------

| You |
-------
Hệ thống báo hiệu đường bộ gồm những gì
----------------------------------------------------------------------------------------------------
| Model |
---------



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Hệ thống báo hiệu đường bộ gồm hiệu lệnh của người điều khiển giao thông; tín hiệu đèn giao thông, biển báo hiệu, vạch kẻ đường, cọc tiêu hoặc tường bảo vệ, rào chắn.
----------------------------------------------------------------------------------------------------

| You |
-------
Quy định về lùi xe gồm những gì?
----------------------------------------------------------------------------------------------------
| Model |
---------



...

- Người điều khiển phải quan sát phía sau.
- Có tín hiệu cần thiết.
- Chỉ khi nào thấy không nguy hiểm mới được lùi.
- Không được lùi xe ở khu vực cấm dừng, trên phần đường dành cho người đi bộ qua đường, nơi đường bộ giao nhau, đường bộ giao
----------------------------------------------------------------------------------------------------

| You |
-------
Khi gặp đèn vàng, tôi được đi tiếp hay phải dừng lại?
----------------------------------------------------------------------------------------------------
| Model |
---------



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


- Nếu đã đi quá vạch dừng thì được đi tiếp.
    - Nếu chưa đi quá vạch dừng thì phải dừng lại trước vạch dừng.
----------------------------------------------------------------------------------------------------

| You |
-------
Các loại xe có quyền ưu tiên đi trước các xe khác khi qua đường giao nhau là gì?
----------------------------------------------------------------------------------------------------
| Model |
---------



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


- Xe chữa cháy đi làm nhiệm vụ
    - Xe quân sự, xe công an đi làm nhiệm vụ khẩn cấp, đoàn xe có xe cảnh sát dẫn đường
    - Xe cứu thương đang thực hiện nhiệm vụ cấp cứu
    - Xe hộ đê, xe đi làm nhiệm
----------------------------------------------------------------------------------------------------

| You |
-------
Tạm biệt


In [ ]:
Hệ thống báo hiệu đường bộ gồm những gì?
Quy định về lùi xe gồm những gì?
Khi gặp đèn vàng, tôi được đi tiếp hay phải dừng lại
Các loại xe có quyền ưu tiên đi trước các xe khác khi qua đường giao nhau là gì?